## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-13-22-50-00 +0000,wsj,Opinion | Seattle Elects Mamdani West,https://www.wsj.com/opinion/seattle-katie-wils...
1,2025-11-13-22-47-51 +0000,nypost,Sledgehammer-wielding maniac allegedly bludgeo...,https://nypost.com/2025/11/13/us-news/sledgeha...
2,2025-11-13-22-43-32 +0000,nyt,Government Shutdown Ends and Federal Workers R...,https://www.nytimes.com/2025/11/13/us/politics...
3,2025-11-13-22-42-08 +0000,nyt,"Some TSA Workers to Get $10,000 Bonus Checks f...",https://www.nytimes.com/2025/11/13/us/politics...
4,2025-11-13-22-40-00 +0000,wsj,The U.S. government is set to reopen after a r...,https://www.wsj.com/politics/policy/federal-go...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
92,trump,41
17,shutdown,22
453,epstein,21
167,new,15
16,government,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
315,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...,150
276,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...,124
312,2025-11-13-01-27-32 +0000,nypost,"Trump signs funding bill to end historic, 43-d...",https://nypost.com/2025/11/12/us-news/house-vo...,112
159,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...,108
285,2025-11-13-04-11-30 +0000,wapo,Longest government shutdown in U.S. history en...,https://www.washingtonpost.com/business/2025/1...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
315,150,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...
276,83,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...
159,57,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...
240,47,2025-11-13-10-30-00 +0000,wsj,Heard on the Street: Borrowing from a builder ...,https://www.wsj.com/economy/housing/builders-c...
137,43,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...
149,40,2025-11-13-16-58-27 +0000,nypost,Disney shares plunge 9% as ABC’s ‘World News T...,https://nypost.com/2025/11/13/business/disney-...
142,38,2025-11-13-17-06-13 +0000,bbc,Starbucks union workers go on strike over pay ...,https://www.bbc.com/news/articles/cwy7r48g0zlo...
313,33,2025-11-13-01-17-00 +0000,wsj,"Rep. Adelita Grijalva (D., Ariz.) became the c...",https://www.wsj.com/politics/policy/petition-f...
135,33,2025-11-13-17-20-59 +0000,bbc,Starmer's chief of staff McSweeney not leaving...,https://www.bbc.com/news/articles/cn8vn4vv87xo...
155,31,2025-11-13-16-32-20 +0000,nypost,‘Big Short’ investor Michael Burry shuts down ...,https://nypost.com/2025/11/13/business/big-sho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
